**Step 1 - Data Understanding**
- shape
- head & tail
- info
- dtypes
- describe
    
**Step 2 - Data Preparation**
- Droping irrelative Columns and Rows
- Identifying duplicated columns
- Renaming columns
- Feature creation
    
**Step 3 - Feature Understanding** (univariate analysis)
- Plotting features distributions
    - Histogram
    - KDE
    - Box plot
    
**Step 4 - Feature Relationship**
- Scatter plot
- Heatmap correlation
- Pair plot
- Groupby comparisons